## Could you please provide me with an illustration of the effect of the measures that you took in order to reduce the "automation"-signature of the driver and the effects each steps head on the driver (I think you said one it effects the headers of the browser). So I would suggest simply illustrate it by pointing out how each steps influences the headers.

1. Setting a Random User-Agent: By rotating the User-Agent with a random one from a predefined list (user_agents), the browser header is modified to appear as if it's coming from different browsers and devices. This variation makes it harder for websites to detect and block automated traffic solely based on User-Agent.

2. Disabling Automation Flags: The lines options.add_experimental_option("useAutomationExtension", False) and options.add_argument("--disable-blink-features=AutomationControlled") disable certain browser features associated with automation. This prevents websites from detecting WebDriver automation through these flags in the browser's header.

3. Disabling Info Bars: The line options.add_argument("--disable-infobars") disables info bars that might indicate the browser is being controlled by automation software. This helps in reducing the detectability of automation.

4. Adding Extension: If an extension is provided (extension_url), it's downloaded and loaded into the browser. This might modify browser behavior, potentially altering the headers and other characteristics of the browser, making it less recognizable as a WebDriver instance.

5. Using Randomized User-Agent in CDP Command: In rotate_user_agent(self), the method execute_cdp_cmd('Network.setUserAgentOverride', {"userAgent": new_user_agent}) sets a new User-Agent using Chrome DevTools Protocol (CDP). This change directly modifies the User-Agent in the browser's headers.

# Install Modules

In [24]:
!pip install selenium

!pip install python-dotenv

!pip install webdriver-manager

!pip install requests

# LinkedIn Login Automation with Selenium

This notebook demonstrates how to automate logging into LinkedIn using Selenium WebDriver with Python. The script will:
- Log into LinkedIn using credentials stored in environment variables.
- Scroll the feed continuously.
- Click the "See new posts" button if it appears.

**Note:** Ensure you have set the environment variables `LINKEDIN_EMAIL` and `LINKEDIN_PASSWORD` with your LinkedIn credentials.

In [34]:
# Import necessary modules
from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager
import random
import time
import os
from dotenv import load_dotenv
load_dotenv()
#load env variable
LINKEDIN_LOGIN_URL= os.environ.get('LINKEDIN_LOGIN_URL')
LINKEDIN_EMAIL= os.environ.get('LINKEDIN_EMAIL')
LINKEDIN_PASSWORD= os.environ.get('LINKEDIN_PASSWORD')
PROXY_PORT= os.environ.get('PROXY_PORT')
PROXY_HOST= os.environ.get('PROXY_HOST')
PROXY_USERNAME= os.environ.get('PROXY_USERNAME')
PROXY_PASSWORD= os.environ.get('PROXY_PASSWORD')
CHROME_EXTENSION_URL= os.environ.get('CHROME_EXTENSION_URL')
FIREFOX_EXTENSION_URL= os.environ.get('FIREFOX_EXTENSION_URL')

## Function to Login to LinkedIn

The following function logs into LinkedIn and scrolls through the feed. It also clicks the "See new posts" button if it appears.

In [35]:
def login_to_linkedin_with_proxy(driver):
    try:
        # Set email here
        email = 'razis928@gmail.com' 
        email_field = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "session_key")))
        email_field.send_keys(email)
        
        # Set password here
        password = 'shah1122'   
        password_field = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "session_password")))
        password_field.send_keys(password)

        login_button = driver.find_element(By.CSS_SELECTOR, "button[type='submit']")
        login_button.click()
        
        WebDriverWait(driver, 10).until(EC.url_contains("https://www.linkedin.com/feed/"))
        
        print("Login successful!")

        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)
            
            # Check if the button exists
            try:
                button = driver.find_element(By.CSS_SELECTOR, "button.artdeco-button.artdeco-button--secondary.mv5.t-14.t-black.t-normal")
                button.click()
                print("Clicked 'See new posts' button.")
            except NoSuchElementException:
                pass
        
    except Exception as e:
        print("Error during login:", e)
        
    finally:
        driver.quit()

## Running the Script

Before running the script, ensure that you have:
- Set up your environment variables `LINKEDIN_EMAIL` and `LINKEDIN_PASSWORD`.
- Installed the necessary packages using `pip install -r requirements.txt`.
- Downloaded the appropriate WebDriver for your browser and added it to your system PATH.

The following cell initializes the WebDriver and calls the `login_to_linkedin_with_proxy` function.

# Custom Firefox WebDriver for Automated Tasks

This notebook demonstrates how to use a custom Firefox WebDriver to automate tasks, such as rotating user agents, managing cookies, and logging into a website. The custom driver includes support for:
- Rotating User-Agent strings.
- Using proxies for HTTP and HTTPS traffic.
- Adding, retrieving, and managing cookies.
- Downloading and installing browser extensions.

**Note:** Ensure you have set the environment variables `LINKEDIN_EMAIL` and `LINKEDIN_PASSWORD` with your LinkedIn credentials.

## Custom Firefox Driver Class

The following class defines a custom Firefox WebDriver that supports rotating User-Agent strings, managing cookies, using proxies, and downloading extensions.

In [36]:
class MyFirefoxDriver(Firefox):
    def __init__(self, user_agents=None,is_enable_auto_flags=True,*args, **kwargs):

        # selopt = {}
        # user_proxy=None
        # if proxy is not None and proxy["host"] and proxy["port"]:

        
        self.extension_name=None
        # if extension_url is not None:
        #     self.extension_name=self.download_extension(extension_url)
        #     self.install_addon(self.extension_name, temporary=True)
        self.user_agents = user_agents or ["Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"]

        options = kwargs.pop('options', None) or FirefoxOptions()
        if is_enable_auto_flags:
            options.set_preference("dom.webdriver.enabled", False)
            options.set_preference('useAutomationExtension', False)
        
        
        super().__init__(*args, options=options, **kwargs)
    def rotate_user_agent(self):
        """Rotates the User-Agent from the pre-defined list by setting a new one."""
        new_user_agent = random.choice(self.user_agents)
        self.profile.set_preference("general.useragent.override", new_user_agent)
        
        # self.execute_cdp_cmd('Network.setUserAgentOverride', {"userAgent": new_user_agent})
        print(f"User-Agent rotated to: {new_user_agent}")
    
    def add_cookie(self, cookie_dict):
        """
        Adds a single cookie to your current session.
        :param cookie_dict: A dictionary object with details for the cookie: 
                            {'name': 'cookie_name', 'value': 'cookie_value', ...}
        """
        super().add_cookie(cookie_dict)

    def add_cookies(self, cookies):
        """
        Adds a list of cookies to your current session.
        :param cookies: A list of dictionaries where each dict contains 
                        details for a single cookie to be added.
        """
        for cookie in cookies:
            self.add_cookie(cookie)

    def get_cookie(self, name):
        """
        Get a single cookie by name. Returns the cookie if found, None if not.
        :param name: Name of the cookie.
        """
        return super().get_cookie(name)

    def get_all_cookies(self):
        """
        Returns a list of all cookies in the current session.
        """
        return super().get_cookies()

    def login_and_save_cookies(self, login_url, login_function, *login_args):
        """
        Perform login and then save the cookies post-login.
        :param login_url: The URL to navigate to for login.
        :param login_function: A function to call to perform login actions.
                               This function should accept a driver instance as its first argument,
                               followed by any *login_args. 
        :param login_args: Arguments necessary to perform the login provided to login_function.
        """
        # Navigate to the login page
        self.get(login_url)
        # Execute the login action, passing this driver instance and any other login args
        login_function(self, *login_args)
        # Save cookies after login
        return self.get_all_cookies()
    def download_extension(self,extension_url):
        response = requests.get(extension_url)
        name='extension.xpi'
        with open(name, 'wb') as f:
            f.write(response.content)
        print("Extension downloaded successfully.")
        return name


def __init__(self, user_agents=None, profile=None, *args, **kwargs): This initializes the Firefox driver instance. It accepts optional arguments for user agents, and browser profile, among others.

rotate_user_agent(self): This method rotates the User-Agent used by the browser. It might be used to simulate different user agents during web scraping or testing to avoid detection or gather diverse data.

add_cookie(self, cookie_dict): Adds a single cookie to the current session. Useful for maintaining session state or authentication across requests.

add_cookies(self, cookies): Adds a list of cookies to the current session. Similar to add_cookie, but for multiple cookies at once.

get_cookie(self, name): Retrieves a single cookie by its name from the current session.

get_all_cookies(self): Retrieves all cookies present in the current session.

login_and_save_cookies(self, login_url, login_function, *login_args): This method automates the login process and then saves the cookies obtained after successful login. It could be handy for web automation tasks requiring authentication.

download_extension(self, extension_url): Downloads a Firefox extension from a given URL. Useful for automation tasks involving extensions, such as testing or customizing browser behavior.

In [ ]:
proxy_host = PROXY_HOST
proxy_port = PROXY_PORT

user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.140 Safari/537.36 Edge/17.17134",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0"
]

# Install the Chrome Driver
service = FirefoxService(GeckoDriverManager().install())

# Download the extension CRX file
extension_url = FIREFOX_EXTENSION_URL




# Initialize the driver with the loaded extension
driver = MyFirefoxDriver(service=service, user_agents=user_agents,is_enable_auto_flags=False)



## Running the Script

Before running the script, ensure that you have:
- Installed the necessary packages using `pip install selenium  requests`.

The following cell initializes the custom Firefox WebDriver and demonstrates how to use the methods defined in the class.

# LinkedIn Automation with Custom Firefox Driver

This notebook demonstrates how to use a custom Firefox WebDriver to automate LinkedIn login, manage cookies, and rotate user agents using Selenium and environment variables. Ensure that you have set the necessary environment variables before running the script.

In [38]:
# Import necessary modules
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import time
import os
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager
import os

## Setting Up User Agents and Firefox Service

Define a list of user agents and set up the Firefox WebDriver service using GeckoDriverManager.

In [40]:
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.140 Safari/537.36 Edge/17.17134",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0"
]

# Install the Firefox Driver
service = FirefoxService(GeckoDriverManager().install())

## Initializing the Custom Firefox Driver

Set up the Firefox profile and initialize the custom driver, then rotate the user agent.

In [41]:


# Initialize the driver with the loaded profile and user agents
driver = MyFirefoxDriver(service=service, user_agents=user_agents,is_enable_auto_flags=False)

driver

SessionNotCreatedException: Message: Expected browser binary location, but unable to find binary in default location, no 'moz:firefoxOptions.binary' capability provided, and no binary flag set on the command line


# Rotate user agents

In [ ]:
driver.rotate_user_agent()

## Logging into LinkedIn and Managing Cookies

Log into LinkedIn using the custom driver, save cookies, and perform additional actions.

In [ ]:
# Login URL
login_url = "https://www.linkedin.com/"

# Perform login and save cookies
cookies = driver.login_and_save_cookies(login_url, login_to_linkedin_with_proxy)

# Add cookies to the driver
driver.add_cookies(cookies)

# Additional actions using the driver, if any
specific_cookie = driver.get_cookie('session_cookie_name')
all_cookies = driver.get_all_cookies()

# Print cookies for demonstration
print(specific_cookie)
print(all_cookies)

## Logging into LinkedIn and Managing Cookies

Log into LinkedIn using the custom driver, save cookies, and perform additional actions.